# Import the dependencies

In [1]:
import re
import random
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import Counter

import tensorflow as tf
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Conv1D, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint

import tensorflow as tf
from transformers import BertConfig
from transformers import RobertaTokenizerFast

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

from matplotlib import pyplot as plt
from matplotlib import rcParams

import sys
import time

sys.path.append("../../../")
from calculus_path_mod.terms.pseudonym import *
from calculus_path_mod.reduction_strategy import *
from calculus_path_mod.terms.arithm_complex_ops import *

C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Py

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Available GPUs:", physical_devices)

Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
sequence_len = 512
batch_size = 64

config = BertConfig(
    vocab_size=9,
    hidden_size=84,
    num_hidden_layers=2,
    num_attention_heads=6,
    intermediate_size=64,
    max_position_embeddings=sequence_len,
    num_labels=2
)

epochs = 30

# Load & Prepare the data

In [4]:
all_data = pd.read_csv("../../article_models_reduct_steps_by_simple_str/data_steps/steps_simple_term_str.csv", delimiter=',')

# leave only unique terms
print(f"Count all terms: {len(all_data)}\n")
all_data = all_data.drop_duplicates(subset="simple_terms").reset_index(drop=True)
print(f"Count original terms: {len(all_data)}\n")

# shuffle the dataset
# all_data = shuffle(all_data, random_state=33).reset_index(drop=True)
all_data = all_data.drop_duplicates(["simple_terms"])

# drop unreducable by LO or RI
print(f"number samples: {len(all_data)}")
all_data = all_data[[x_ != 1000 for x_ in all_data["RI_steps_num"]]].reset_index(drop=True)
all_data = all_data[[x_ != 1000 for x_ in all_data["LO_steps_num"]]].reset_index(drop=True)
print(f"number samples only reducable: {len(all_data)}\n")

print(f"max RI steps count: {max(all_data['RI_steps_num'])}")
print(f"max LO steps count: {max(all_data['LO_steps_num'])}")

x_test = all_data["simple_terms"].tolist()
# RI has fewer steps -> 1
# Otherwise 0
y_test = [1 if los > ris else 0 for los, ris in zip(all_data["LO_steps_num"].tolist(), all_data["RI_steps_num"].tolist())]
y_ri_test = all_data["LO_steps_num"].tolist()
y_lo_test = all_data["RI_steps_num"].tolist()

print(f"Count TESTING samples: {len(y_test)}")

Count all terms: 4251

Count original terms: 4251

number samples: 4251
number samples only reducable: 4220

max RI steps count: 386
max LO steps count: 219
Count TESTING samples: 4220


In [5]:
all_data = pd.read_csv("../../article_models_reduct_steps_by_simple_str/data_steps/steps_simple_term_str_extended.csv", delimiter=',')

# leave only unique terms
print(f"Count all terms: {len(all_data)}\n")
all_data = all_data.drop_duplicates(subset="simple_terms").reset_index(drop=True)
print(f"Count original terms: {len(all_data)}\n")

# shuffle the dataset
# all_data = shuffle(all_data, random_state=33).reset_index(drop=True)
all_data = all_data.drop_duplicates(["simple_terms"])

# drop unreducable by LO or RI
print(f"number samples: {len(all_data)}")
all_data = all_data[[x_ != 1000 for x_ in all_data["RI_steps_num"]]].reset_index(drop=True)
all_data = all_data[[x_ != 1000 for x_ in all_data["LO_steps_num"]]].reset_index(drop=True)
print(f"number samples only reducable: {len(all_data)}\n")

print(f"max RI steps count: {max(all_data['RI_steps_num'])}")
print(f"max LO steps count: {max(all_data['LO_steps_num'])}")

x_train = all_data["simple_terms"].tolist()
# RI has fewer steps -> 1
# Otherwise 0
y_train = [1 if los > ris else 0 for los, ris in zip(all_data["LO_steps_num"].tolist(), all_data["RI_steps_num"].tolist())]
y_ri_train = all_data["LO_steps_num"].tolist()
y_lo_train = all_data["RI_steps_num"].tolist()

print(f"Count TRAINING samples: {len(y_train)}")

Count all terms: 44568

Count original terms: 44568

number samples: 44568
number samples only reducable: 42469

max RI steps count: 400
max LO steps count: 308
Count TRAINING samples: 42469


In [6]:
tokenizer = RobertaTokenizerFast.from_pretrained("../../transformers_to_lo_steps_prediciton/fine_models", max_len=sequence_len)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
x_train = [x_.replace("@x.", "y").replace(" ", "") for x_ in x_train]
x_test = [x_.replace("@x.", "y").replace(" ", "") for x_ in x_test]

train_df = pd.DataFrame({"term_str": x_train, "is_ri_best": y_train})
test_df = pd.DataFrame({"term_str": x_test, "is_ri_best": y_test})

def preprocess(example):
    # Tokenize the prompt
    tokenized_texts = tokenizer(example['term_str'].to_list(), truncation=True, padding='max_length', max_length=sequence_len, return_tensors="tf")
    labels = tf.convert_to_tensor(example["is_ri_best"])
    return tokenized_texts, labels


tokenized_train_data = preprocess(train_df)
tokenized_test_data = preprocess(test_df)

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_train_data[0]), tokenized_train_data[1])).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_test_data[0]), tokenized_test_data[1])).batch(batch_size)

In [9]:
def calculate_accuracy(actual_labels, predicted_labels):
    correct_predictions = sum(1 for actual, predicted in zip(actual_labels, predicted_labels) if actual == predicted)
    total_predictions = len(actual_labels)
    accuracy = correct_predictions / total_predictions
    return accuracy


from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, classes, class_labels, normalize=False, title=None, cmap=plt.cm.Blues):
    """
    This function plots the confusion matrix of a classification model.

    Args:
        y_true (numpy.ndarray): The ground truth labels.
        y_pred (numpy.ndarray): The predicted labels.
        classes (list): The list of class labels.
        class_labels: The list of class names.
        normalize (bool, optional): Whether to normalize the confusion matrix. Defaults to False.
        title (str, optional): The title of the plot. Defaults to None.
        cmap (matplotlib.colors.Colormap, optional): The colormap to use for the plot. Defaults to plt.cm.Blues.
    """

    cm = confusion_matrix(y_true, y_pred, labels=classes)

    if normalize:
        cm = confusion_matrix(y_true, y_pred, labels=classes).astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.set_xticks(np.arange(len(classes)))
    ax.set_yticks(np.arange(len(classes)))
    ax.set_xticklabels(class_labels)
    ax.set_yticklabels(class_labels)
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
    ax.set_title(title)
    fig.colorbar(im)

    thresh = cm.max() / 2.
    for i in range(len(classes)):
        for j in range(len(classes)):
            ij = float(cm[i, j])
            ax.text(j, i, f"{ij:.2f}", ha='center', va='center', color='white' if ij > thresh else 'black')

    plt.tight_layout()
    plt.show()

# The best & worst possible steps

In [10]:
y_test_best_sum = 0
for lo_, ri_ in zip(y_lo_test, y_ri_test):
    y_test_best_sum += lo_ if lo_ <= ri_ else ri_
y_test_best_avg = y_test_best_sum / len(y_lo_test)

y_train_best_sum = 0
for lo_, ri_ in zip(y_lo_train, y_ri_train):
    y_train_best_sum += lo_ if lo_ <= ri_ else ri_
y_train_best_avg = y_train_best_sum / len(y_lo_train)


y_test_worst_sum = 0
for lo_, ri_ in zip(y_lo_test, y_ri_test):
    y_test_worst_sum += lo_ if lo_ > ri_ else ri_
y_test_worst_avg = y_test_worst_sum / len(y_lo_test)

y_train_worst_sum = 0
for lo_, ri_ in zip(y_lo_train, y_ri_train):
    y_train_worst_sum += lo_ if lo_ > ri_ else ri_
y_train_worst_avg = y_train_worst_sum / len(y_lo_train)


y_test_worst_LO_sum = np.sum(y_lo_test)
y_test_worst_LO_avg = np.mean(y_lo_test)

y_train_worst_LO_sum = np.sum(y_lo_train)
y_train_worst_LO_avg = np.mean(y_lo_train)


y_test_worst_RI_sum = np.sum(y_ri_test)
y_test_worst_RI_avg = np.mean(y_ri_test)

y_train_worst_RI_sum = np.sum(y_ri_train)
y_train_worst_RI_avg = np.mean(y_ri_train)


print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Test WORST steps: avg={y_test_worst_avg:.3f}, sum={y_test_worst_sum}")
print(f"Train WORST steps: avg={y_train_worst_avg:.3f}, sum={y_train_worst_sum}\n")

print(f"Test WORST LO steps: avg={y_test_worst_LO_avg:.3f}, sum={y_test_worst_LO_sum}")
print(f"Train WORST LO steps: avg={y_train_worst_LO_avg:.3f}, sum={y_train_worst_LO_sum}\n")

print(f"Test WORST RI steps: avg={y_test_worst_RI_avg:.3f}, sum={y_test_worst_RI_sum}")
print(f"Train WORST RI steps: avg={y_train_worst_RI_avg:.3f}, sum={y_train_worst_RI_sum}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Test WORST steps: avg=23.878, sum=100764
Train WORST steps: avg=24.455, sum=1038575

Test WORST LO steps: avg=22.375, sum=94422
Train WORST LO steps: avg=22.175, sum=941755

Test WORST RI steps: avg=16.753, sum=70698
Train WORST RI steps: avg=15.590, sum=662085



In [11]:
def calc_steps_accuracy(y_predictions, y_lo_steps, y_ri_steps, threshold=0.5):
    y_steps_sum = 0
    for lo_, ri_, pred_ in zip(y_lo_steps, y_ri_steps, y_predictions):
        y_steps_sum += lo_ if pred_ < threshold else ri_
    y_steps_avg = y_steps_sum / len(y_lo_steps)
    return y_steps_sum, y_steps_avg

# Test 160 epochs trained model on all terms

In [11]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 46s 69ms/step


In [20]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 160ep, 0.5th")
y_test_sum_160_05, y_test_avg_160_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_160_05, y_train_avg_160_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_160_05:.3f}, sum={y_test_sum_160_05}")
print(f"Train steps: avg={y_train_avg_160_05:.3f}, sum={y_train_sum_160_05}\n")

print(f"Models results 160ep, 0.35th")
y_test_sum_160_035, y_test_avg_160_035 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_160_035, y_train_avg_160_035 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_160_035:.3f}, sum={y_test_sum_160_035}")
print(f"Train steps: avg={y_train_avg_160_035:.3f}, sum={y_train_sum_160_035}\n")

print(f"Models results 160ep, 0.15th")
y_test_sum_160_015, y_test_avg_160_015 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_160_015, y_train_avg_160_015 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_160_015:.3f}, sum={y_test_sum_160_015}")
print(f"Train steps: avg={y_train_avg_160_015:.3f}, sum={y_train_sum_160_015}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 160ep, 0.5th
Test steps: avg=22.072, sum=93142
Train steps: avg=23.953, sum=1017261

Models results 160ep, 0.35th
Test steps: avg=21.980, sum=92754
Train steps: avg=23.990, sum=1018851

Models results 160ep, 0.15th
Test steps: avg=21.522, sum=90824
Train steps: avg=23.865, sum=1013519



# Test 160 epochs trained, best f1 model on all terms

In [21]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_best_f1.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 46s 69ms/step


In [22]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 160ep f1, 0.5th")
y_test_sum_160_f1_05, y_test_avg_160_f1_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_160_f1_05, y_train_avg_160_f1_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_160_f1_05:.3f}, sum={y_test_sum_160_f1_05}")
print(f"Train steps: avg={y_train_avg_160_f1_05:.3f}, sum={y_train_sum_160_f1_05}\n")

print(f"Models results 160ep f1, 0.35th")
y_test_sum_160_f1_035, y_test_avg_160_f1_035 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_160_f1_035, y_train_avg_160_f1_035 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_160_f1_035:.3f}, sum={y_test_sum_160_f1_035}")
print(f"Train steps: avg={y_train_avg_160_f1_035:.3f}, sum={y_train_sum_160_f1_035}\n")

print(f"Models results 160ep f1, 0.15th")
y_test_sum_160_f1_015, y_test_avg_160_f1_015 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_160_f1_015, y_train_avg_160_f1_015 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_160_f1_015:.3f}, sum={y_test_sum_160_f1_015}")
print(f"Train steps: avg={y_train_avg_160_f1_015:.3f}, sum={y_train_sum_160_f1_015}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 160ep f1, 0.5th
Test steps: avg=22.173, sum=93570
Train steps: avg=23.591, sum=1001904

Models results 160ep f1, 0.35th
Test steps: avg=21.850, sum=92205
Train steps: avg=23.650, sum=1004392

Models results 160ep f1, 0.15th
Test steps: avg=21.261, sum=89722
Train steps: avg=23.389, sum=993316



# Test 90 epochs trained model on reducible terms

In [23]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_only_reducable.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 46s 69ms/step


In [24]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, only reducible, 0.5th")
y_test_sum_90_red_05, y_test_avg_90_red_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_90_red_05, y_train_avg_90_red_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_90_red_05:.3f}, sum={y_test_sum_90_red_05}")
print(f"Train steps: avg={y_train_avg_90_red_05:.3f}, sum={y_train_sum_90_red_05}\n")

print(f"Models results 90ep, only reducible, 0.35th")
y_test_sum_90_red_035, y_test_avg_90_red_035 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_90_red_035, y_train_avg_90_red_035 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_90_red_035:.3f}, sum={y_test_sum_90_red_035}")
print(f"Train steps: avg={y_train_avg_90_red_035:.3f}, sum={y_train_sum_90_red_035}\n")

print(f"Models results 90ep, only reducible, 0.15th")
y_test_sum_90_red_015, y_test_avg_90_red_015 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_90_red_015, y_train_avg_90_red_015 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_90_red_015:.3f}, sum={y_test_sum_90_red_015}")
print(f"Train steps: avg={y_train_avg_90_red_015:.3f}, sum={y_train_sum_90_red_015}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, only reducible, 0.5th
Test steps: avg=21.961, sum=92676
Train steps: avg=23.575, sum=1001201

Models results 90ep, only reducible, 0.35th
Test steps: avg=21.818, sum=92072
Train steps: avg=23.661, sum=1004850

Models results 90ep, only reducible, 0.15th
Test steps: avg=21.277, sum=89790
Train steps: avg=23.558, sum=1000475



# Test 90 epochs trained best f1 model on reducible terms

In [25]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_best_f1_only_reducable.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 46s 70ms/step


In [26]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, only reducible, 0.5th")
y_test_sum_90_red_f1_05, y_test_avg_90_red_f1_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_90_red_f1_05, y_train_avg_90_red_f1_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_90_red_f1_05:.3f}, sum={y_test_sum_90_red_05}")
print(f"Train steps: avg={y_train_avg_90_red_f1_05:.3f}, sum={y_train_sum_90_red_05}\n")

print(f"Models results 90ep, only reducible, 0.35th")
y_test_sum_90_red_f1_035, y_test_avg_90_red_f1_035 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_90_red_f1_035, y_train_avg_90_red_f1_035 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_90_red_f1_035:.3f}, sum={y_test_sum_90_red_f1_035}")
print(f"Train steps: avg={y_train_avg_90_red_f1_035:.3f}, sum={y_train_sum_90_red_f1_035}\n")

print(f"Models results 90ep, only reducible, 0.15th")
y_test_sum_90_red_f1_015, y_test_avg_90_red_f1_015 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_90_red_f1_015, y_train_avg_90_red_f1_015 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_90_red_f1_015:.3f}, sum={y_test_sum_90_red_f1_015}")
print(f"Train steps: avg={y_train_avg_90_red_f1_015:.3f}, sum={y_train_sum_90_red_f1_015}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, only reducible, 0.5th
Test steps: avg=22.058, sum=92676
Train steps: avg=23.591, sum=1001201

Models results 90ep, only reducible, 0.35th
Test steps: avg=21.943, sum=92599
Train steps: avg=23.635, sum=1003738

Models results 90ep, only reducible, 0.15th
Test steps: avg=21.423, sum=90404
Train steps: avg=23.409, sum=994177



# Test 90 epoch with sample_weight
 sample_weight = (abs(ri_steps - lo_steps) / (max(ri_steps, lo_steps) + mu) + mu), where mu = 0.001

In [11]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_red_weighted.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 34s 51ms/step


In [14]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted, 0.5th")
y_test_sum_90_red_w_05, y_test_avg_90_red_w_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_90_red_w_05, y_train_avg_90_red_w_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_90_red_w_05:.3f}, sum={y_test_sum_90_red_w_05}")
print(f"Train steps: avg={y_train_avg_90_red_w_05:.3f}, sum={y_train_sum_90_red_w_05}\n")

print(f"Models results 90ep, red weighted, 0.35th")
y_test_sum_90_red_w_035, y_test_avg_90_red_w_035 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_90_red_w_035, y_train_avg_90_red_w_035 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_90_red_w_035:.3f}, sum={y_test_sum_90_red_w_035}")
print(f"Train steps: avg={y_train_avg_90_red_w_035:.3f}, sum={y_train_sum_90_red_w_035}\n")

print(f"Models results 90ep, red weighted, 0.15th")
y_test_sum_90_red_w_015, y_test_avg_90_red_w_015 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_90_red_w_015, y_train_avg_90_red_w_015 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_90_red_w_015:.3f}, sum={y_test_sum_90_red_w_015}")
print(f"Train steps: avg={y_train_avg_90_red_w_015:.3f}, sum={y_train_sum_90_red_w_015}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted, 0.5th
Test steps: avg=21.420, sum=90393
Train steps: avg=24.162, sum=1026129

Models results 90ep, red weighted, 0.35th
Test steps: avg=21.289, sum=89840
Train steps: avg=24.163, sum=1026174

Models results 90ep, red weighted, 0.15th
Test steps: avg=21.007, sum=88648
Train steps: avg=24.040, sum=1020935



# Test 90 epoch with sample_weight BEST F1
 sample_weight = (abs(ri_steps - lo_steps) / (max(ri_steps, lo_steps) + mu) + mu), where mu = 0.001

In [12]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_best_f1_red_weighted.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 34s 51ms/step


In [13]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted, 0.5th
Test steps: avg=21.877, sum=92322
Train steps: avg=24.029, sum=1020475

Models results 90ep, red weighted, 0.35th
Test steps: avg=21.779, sum=91906
Train steps: avg=24.116, sum=1024169

Models results 90ep, red weighted, 0.15th
Test steps: avg=21.551, sum=90945
Train steps: avg=24.085, sum=1022882



# Test 90 epoch with sample_weight
 if ri_steps < lo_steps:
    sample_weight = abs(ri_steps - lo_steps) / (max(ri_steps, lo_steps),
 else:
    sample_weight = 0.001

In [13]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_red_weighted_v2.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 47s 71ms/step


In [14]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v2, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v2, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v2, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v2, 0.5th
Test steps: avg=16.755, sum=70704
Train steps: avg=15.591, sum=662125

Models results 90ep, red weighted v2, 0.35th
Test steps: avg=16.754, sum=70702
Train steps: avg=15.590, sum=662108

Models results 90ep, red weighted v2, 0.15th
Test steps: avg=16.754, sum=70700
Train steps: avg=15.590, sum=662094



# Test 90 epoch with sample_weight BEST F1
 if ri_steps < lo_steps:
    sample_weight = abs(ri_steps - lo_steps) / (max(ri_steps, lo_steps),
 else:
    sample_weight = 0.001

In [15]:
with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_best_f1_red_weighted_v2.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 47s 70ms/step


In [16]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v2, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v2, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v2, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v2, 0.5th
Test steps: avg=16.791, sum=70859
Train steps: avg=15.968, sum=678127

Models results 90ep, red weighted v2, 0.35th
Test steps: avg=16.783, sum=70825
Train steps: avg=15.960, sum=677819

Models results 90ep, red weighted v2, 0.15th
Test steps: avg=16.774, sum=70785
Train steps: avg=15.910, sum=675684



# Test 90 epoch with sample_weight
 if ri_steps > lo_steps:
    sample_weight = abs(ri_steps - lo_steps) / (max(ri_steps, lo_steps),
 else:
    sample_weight = 0.001

In [12]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_red_weighted_v3.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 47s 71ms/step


In [13]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v3, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v3, 0.5th
Test steps: avg=22.375, sum=94422
Train steps: avg=22.175, sum=941755

Models results 90ep, red weighted v3, 0.35th
Test steps: avg=22.375, sum=94422
Train steps: avg=22.305, sum=947291

Models results 90ep, red weighted v3, 0.15th
Test steps: avg=22.375, sum=94422
Train steps: avg=22.313, sum=947627



# Test 90 epoch with sample_weight BEST F1
 if ri_steps > lo_steps:
    sample_weight = abs(ri_steps - lo_steps) / (max(ri_steps, lo_steps),
 else:
    sample_weight = 0.001

In [14]:
with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_best_f1_red_weighted_v3.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 62s 93ms/step


In [15]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v3, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v3, 0.5th
Test steps: avg=22.375, sum=94422
Train steps: avg=22.175, sum=941755

Models results 90ep, red weighted v3, 0.35th
Test steps: avg=22.375, sum=94422
Train steps: avg=22.175, sum=941755

Models results 90ep, red weighted v3, 0.15th
Test steps: avg=22.375, sum=94422
Train steps: avg=22.175, sum=941755



# Test 90 epoch with sample_weight

* abs_diff = np.abs(ri_steps - li_steps)
* abs_diff += 2
* return np.log(abs_diff) - 0.6

In [14]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_red_weighted_v4.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 32s 48ms/step


In [15]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v3, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v3, 0.5th
Test steps: avg=21.524, sum=90831
Train steps: avg=23.906, sum=1015266

Models results 90ep, red weighted v3, 0.35th
Test steps: avg=21.317, sum=89958
Train steps: avg=23.957, sum=1017420

Models results 90ep, red weighted v3, 0.15th
Test steps: avg=21.037, sum=88778
Train steps: avg=23.733, sum=1007918



# Test 90 epoch with sample_weight BEST F1

* abs_diff = np.abs(ri_steps - li_steps)
* abs_diff += 2
* return np.log(abs_diff) - 0.6

In [16]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_best_f1_red_weighted_v4.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 35s 52ms/step


In [17]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v3, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v3, 0.5th
Test steps: avg=21.858, sum=92240
Train steps: avg=23.882, sum=1014260

Models results 90ep, red weighted v3, 0.35th
Test steps: avg=21.586, sum=91095
Train steps: avg=23.923, sum=1015972

Models results 90ep, red weighted v3, 0.15th
Test steps: avg=21.006, sum=88647
Train steps: avg=23.695, sum=1006318



# Test 180 epoch with sample_weight
sample_weight = np.abs(ri_steps - li_steps)

In [13]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_red_weighted_v6.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 44s 66ms/step


In [14]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v3, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v3, 0.5th
Test steps: avg=21.293, sum=89858
Train steps: avg=23.953, sum=1017259

Models results 90ep, red weighted v3, 0.35th
Test steps: avg=21.021, sum=88710
Train steps: avg=23.916, sum=1015672

Models results 90ep, red weighted v3, 0.15th
Test steps: avg=20.630, sum=87060
Train steps: avg=23.667, sum=1005133



# Test 180 epoch with sample_weight, BEST F1
sample_weight = np.abs(ri_steps - li_steps)\

In [13]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_best_f1_red_weighted_v6.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 44s 66ms/step


In [14]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v3, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v3, 0.5th
Test steps: avg=21.445, sum=90498
Train steps: avg=23.759, sum=1009008

Models results 90ep, red weighted v3, 0.35th
Test steps: avg=21.139, sum=89205
Train steps: avg=23.719, sum=1007338

Models results 90ep, red weighted v3, 0.15th
Test steps: avg=20.624, sum=87034
Train steps: avg=23.388, sum=993258



# Test 90 epoch with sample_weight
sample_weight = np.abs(ri_steps - li_steps) + 1

In [12]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_red_weighted_v7.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 33s 49ms/step


In [13]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v3, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v3, 0.5th
Test steps: avg=21.394, sum=90284
Train steps: avg=23.629, sum=1003486

Models results 90ep, red weighted v3, 0.35th
Test steps: avg=21.035, sum=88766
Train steps: avg=23.613, sum=1002818

Models results 90ep, red weighted v3, 0.15th
Test steps: avg=20.492, sum=86478
Train steps: avg=23.098, sum=980931



# Test 90 epoch with sample_weight, BEST F1
sample_weight = np.abs(ri_steps - li_steps) + 1

In [14]:
# load the model
from transformers import TFBertModel

with open("./fine_models/model_ri.json", "r") as file:
    loaded_model_json = file.read()

model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={"TFBertModel": TFBertModel})
model.load_weights('./fine_models/model_ri_best_f1_red_weighted_v7.h5')


y_test_pred = model.predict(test_dataset)
y_train_pred = model.predict(train_dataset)

664/664 [==============================] - 32s 49ms/step


In [15]:
print(f"Test BEST steps: avg={y_test_best_avg:.3f}, sum={y_test_best_sum}")
print(f"Train BEST steps: avg={y_train_best_avg:.3f}, sum={y_train_best_sum}\n")

print(f"Models results 90ep, red weighted v3, 0.5th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.5)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.5)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.35th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.35)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.35)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

print(f"Models results 90ep, red weighted v3, 0.15th")
y_test_sum_05, y_test_avg_05 = calc_steps_accuracy(y_test_pred, y_lo_test, y_ri_test, threshold=0.15)
y_train_sum_05, y_train_avg_05 = calc_steps_accuracy(y_train_pred, y_lo_train, y_ri_train, threshold=0.15)
print(f"Test steps: avg={y_test_avg_05:.3f}, sum={y_test_sum_05}")
print(f"Train steps: avg={y_train_avg_05:.3f}, sum={y_train_sum_05}\n")

Test BEST steps: avg=15.250, sum=64356
Train BEST steps: avg=13.310, sum=565265

Models results 90ep, red weighted v3, 0.5th
Test steps: avg=21.789, sum=91951
Train steps: avg=23.463, sum=996460

Models results 90ep, red weighted v3, 0.35th
Test steps: avg=21.509, sum=90767
Train steps: avg=23.394, sum=993502

Models results 90ep, red weighted v3, 0.15th
Test steps: avg=20.743, sum=87535
Train steps: avg=22.814, sum=968902

